# Question 1

### Simulate GARCH(1,1) data

In [ ]:
import numpy as np

def simulate_garch(T, omega_1, omega_2, omega_3):
    np.random.seed(42)  # For reproducibility
    r = np.zeros(T)
    h = np.zeros(T)
    r[0] = np.random.normal(0, np.sqrt(omega_1 / (1 - omega_2 - omega_3)))
    h[0] = omega_1 / (1 - omega_2 - omega_3)
    
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        r[t] = np.random.normal(0, np.sqrt(h[t]))
        
    return r, h

# Parameters (example values)
omega_1 = 0.1
omega_2 = 0.1
omega_3 = 0.8

# Simulate data
T = 1000  # Number of time points
r, h = simulate_garch(T, omega_1, omega_2, omega_3)


### Log-Likelihood & Log-Posterior Functions

The likelihood function for the GARCH model parameters $\omega_1, \omega_2, \omega_3$ given returns $r$ is defined as:
$$
l(\omega_1, \omega_2, \omega_3 \mid r) \propto \prod_{t=1}^T h_t^{-\frac{1}{2}} \exp\left(-\frac{1}{2} \sum_{t=1}^T \frac{r_t^2}{h_t}\right)
$$

The posterior distribution, incorporating independent truncated Normal priors for the parameters, is given by:
$$
\pi(\omega_1, \omega_2, \omega_3 \mid r) \propto \exp\left(-\frac{1}{2} \left(\frac{\omega_1^2}{\sigma^2(\omega_1)} + \frac{\omega_2^2}{\sigma^2(\omega_2)} + \frac{\omega_3^2}{\sigma^2(\omega_3)}\right)\right) \prod_{t=1}^T h_t^{-\frac{1}{2}} \exp\left(-\frac{1}{2} \sum_{t=1}^T \frac{r_t^2}{h_t}\right)
$$

This gives us both the log-likelihood expression and the log-posterior expression:
$$
\log l(\omega_1, \omega_2, \omega_3 \mid r) = -\frac{1}{2} \sum_{t=1}^T \left( \log h_t + \frac{r_t^2}{h_t}\right)
$$

$$
\log \pi(\omega_1, \omega_2, \omega_3 \mid r) = -\frac{1}{2} \left(\frac{\omega_1^2}{\sigma^2(\omega_1)} + \frac{\omega_2^2}{\sigma^2(\omega_2)} + \frac{\omega_3^2}{\sigma^2(\omega_3)}\right) -\frac{1}{2} \sum_{t=1}^T \left( \log h_t + \frac{r_t^2}{h_t}\right)
$$


In [ ]:
def log_likelihood(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    h[0] = omega_1 / (1 - omega_2 - omega_3) if (omega_2 + omega_3) < 1 else 100000  # large number for stability
    
    log_lik = 0
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        log_lik -= 0.5 * (np.log(h[t]) + r[t]**2 / h[t])
    
    return log_lik

def log_posterior(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    log_prior = -0.5 * (omega_1**2 / sigma_1**2 + omega_2**2 / sigma_2**2 + omega_3**2 / sigma_3**2)
    return log_likelihood(r, omega_1, omega_2, omega_3) + log_prior


### Implement the RWMH Sampler

In [ ]:
# To be done

# Question 2

### Gradient Computations to find Control Variates

The gradient of the log-posterior $\ln \pi$ with respect to each parameter $ \omega_i $ involves contributions from the prior and the likelihood:
$$
\frac{\partial \ln \pi}{\partial \omega_i} = -\frac{\omega_i}{\sigma^2(\omega_i)} - \frac{1}{2} \sum_{t=1}^T \left( \frac{1}{h_t} \frac{\partial h_t}{\partial \omega_i} - \frac{r_t^2}{h_t^2} \frac{\partial h_t}{\partial \omega_i} \right), \quad i = 1, 2, 3
$$

The derivatives of the volatility equation $ h_t $ with respect to the parameters are defined as:
$$
\frac{\partial h_t}{\partial \omega_1} = \frac{1 - \omega_3^{t-1}}{1 - \omega_3} \quad \text{(assuming $ \omega_3 \neq 1 $)}
$$

$$
\frac{\partial h_t}{\partial \omega_2} = r_{t-1}^2 + \omega_3 \frac{\partial h_{t-1}}{\partial \omega_2} \quad \text{for } t > 1 \text{, and 0 else}
$$

$$
\frac{\partial h_t}{\partial \omega_3} = h_{t-1} + \omega_3 \frac{\partial h_{t-1}}{\partial \omega_3} \quad \text{for } t > 1 \text{, and 0 else}
$$


**Error in the article ? Should be $\omega_3^t$:**

*Objective:*
Compute the derivative of $h_t= \omega_1 + \omega_2 r_{t-1}^2 + \omega_3 h_{t-1}$ with respect to $\omega_1$, $\frac{\partial h_t}{\partial \omega_1}$.

*Initial Setup:*
Since $\omega_1$ is a constant addition at each time step $t$,
$ \frac{\partial \omega_1}{\partial \omega_1} = 1 $
and since $r_{t-1}^2$ does not depend on $\omega_1$,
$ \frac{\partial (r_{t-1}^2)}{\partial \omega_1} = 0 $

*Recursive Derivation:*
Starting from the base case at $t=0$, assuming $h_0$ is either non-dependent on $\omega_1$ or set by an initial condition (making it a constant):
$ h_1 = \omega_1 + \omega_2 r_0^2 + \omega_3 h_0 $ gives 
$ \frac{\partial h_1}{\partial \omega_1} = 1 $

At $t=2$:
$ h_2 = \omega_1 + \omega_2 r_1^2 + \omega_3 h_1 $ gives
$ \frac{\partial h_2}{\partial \omega_1} = 1 + \omega_3 \cdot \frac{\partial h_1}{\partial \omega_1} = 1 + \omega_3 $

Proceeding recursively,
$ h_3 = \omega_1 + \omega_2 r_2^2 + \omega_3 h_2 $ gives 
$ \frac{\partial h_3}{\partial \omega_1} = 1 + \omega_3 \cdot \frac{\partial h_2}{\partial \omega_1} = 1 + \omega_3 + \omega_3^2 $

$$\vdots$$

$ \frac{\partial h_t}{\partial \omega_1} = 1 + \omega_3 + \omega_3^2 + \ldots + \omega_3^{t-1} $

Summation as a geometric series for $\omega_3 \neq 1$ yields:
$$ \frac{\partial h_t}{\partial \omega_1} = \frac{1 - \omega_3^t}{1 - \omega_3} $$


In [ ]:
def compute_h_t_derivatives(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    grad_h_omega1 = np.zeros(T)
    grad_h_omega2 = np.zeros(T)
    grad_h_omega3 = np.zeros(T)

    # Initial volatility
    h[0] = omega_1  

    # Compute h and its derivatives
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        
        # Derivative of h_t with respect to omega_1
        grad_h_omega1[t] = (1 - omega_3**(t)) / (1 - omega_3) if omega_3 != 1 else t  # Using the geometric series

        # Derivative of h_t with respect to omega_2
        grad_h_omega2[t] = r[t-1]**2 + omega_3 * grad_h_omega2[t-1]

        # Derivative of h_t with respect to omega_3
        grad_h_omega3[t] = h[t-1] + omega_3 * grad_h_omega3[t-1]

    return h, grad_h_omega1, grad_h_omega2, grad_h_omega3

def compute_log_posterior_gradients(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    h, grad_h_omega1, grad_h_omega2, grad_h_omega3 = compute_h_t_derivatives(r, omega_1, omega_2, omega_3)
    T = len(r)
    gradients = np.zeros(3)

    # Compute the gradient of the log-posterior for each parameter
    gradients[0] = -omega_1 / sigma_1**2 - 0.5 * np.sum((1 / h) * grad_h_omega1 - (r**2 / h**2) * grad_h_omega1)
    gradients[1] = -omega_2 / sigma_2**2 - 0.5 * np.sum((1 / h) * grad_h_omega2 - (r**2 / h**2) * grad_h_omega2)
    gradients[2] = -omega_3 / sigma_3**2 - 0.5 * np.sum((1 / h) * grad_h_omega3 - (r**2 / h**2) * grad_h_omega3)

    return gradients